In [1]:
import sys
sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# del os.environ["WANDB_DISABLED"]

In [ ]:
import analyze_utils

checkpoint_folder = '/home/jxm3/research/retrieval/inversion/saves/8631b1c05efebde3077d16c5b99f6d5e/dc72e8b9c01bd27b0ed1c2def90bcee5'

trainer = analyze_utils.load_inversion_model_and_trainer(checkpoint_folder)

In [ ]:

os.environ["WANDB_DISABLED"] = "true"
max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)

print("[6] calling evaluate()")
metrics = trainer.evaluate()
metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

print("metrics:", metrics)

In [ ]:
from utils import embed_all_tokens

device = torch.device('cuda')
trainer.model.embedded_tokens = embed_all_tokens(trainer.model, embedder_tokenizer).to(device)

In [ ]:
trainer.args.per_device_eval_batch_size = 128

In [ ]:
from tqdm.notebook import tqdm

data = []
for alpha in tqdm([0.9, 0.99, .999, 0.9999]): # , 0.999, 0.9999]):
    for temperature in [20, 10, 2, 1, 1e-1, 1e-2]:
        trainer.model.token_decode_alpha = alpha
        trainer.model.token_decode_temperature = temperature
        # print('** temperature **', temperature)
        metrics = trainer.evaluate()
        metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
        metrics["alpha"] = alpha
        metrics["temperature"] = temperature
        data.append(metrics)


In [ ]:
import pandas as pd
import seaborn as sns

full_df = pd.DataFrame(data)
full_df.head()

In [ ]:
# baseline eval_bleu_score 7.307456928008692

In [ ]:
full_df.pivot('alpha', 'temperature', 'eval_bleu_score')

In [ ]:
df = full_df.pivot('alpha', 'temperature', 'eval_bleu_score')
sns.heatmap(data=df, cmap='viridis')